In [3]:
import pandas
import xgboost as xgb
import numpy as np

df = pandas.read_csv('osu18_cerenkov_feat_mat.tsv', sep='\t')
df.head()

labels = df['label']
del df['label']

names = df['name']
del df['name']

In [4]:
trainDf = df[:10000]
testDf = df[10000:]

In [5]:
dtrain = xgb.DMatrix(trainDf, label=labels[:10000])
dtest = xgb.DMatrix(testDf, label=labels[10000:])



evallist = [(dtest, 'eval'), (dtrain, 'train')]


# param_dist = dict(max_depth=[7],
#                   learning_rate=[0.1],
#                   n_estimators=[40], 
#                   gamma=[10],
#                   scale_pos_weight=[1],
#                   base_score=[rare_event_rate],
#                   subsample=[1])



In [10]:
param = dict(max_depth=7,
            learning_rate=0.1,
            n_estimators=10,
            gamma=10,
            scale_pos_weight=1,
            base_score=0.5,
            subsample=1)

num_round = 5

bst = xgb.train(param, dtrain, num_round, evallist)

[01:00:14] WARNING: /workspace/src/learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-rmse:nan	train-rmse:0.46020
[1]	eval-rmse:nan	train-rmse:0.42525
[2]	eval-rmse:nan	train-rmse:0.39490
[3]	eval-rmse:nan	train-rmse:0.36848
[4]	eval-rmse:nan	train-rmse:0.34555
